In [1]:
from utils import load_model_and_tokenizer, call_model, gather_logprobs

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model_name = "meta-llama/Llama-2-13b-chat-hf"
# model_name = "meta-llama/Llama-2-7b-chat"
# model_name = "gpt2"
model, tokenizer = load_model_and_tokenizer(model_name)
device = model.device

Loading checkpoint shards: 100%|██████████| 3/3 [00:29<00:00,  9.90s/it]


In [9]:
prompt =\
"""
Goal
Make a classifier for whether a given piece of text can meaningfully be classified as true or false. More specifically, classify whether the the prefix of a document ending at a token t can be classified as true or false (whether it's “truth-apt”). Then we can use such a classifier to figure out which tokens to use an ELK probe on for a given piece of text (especially a debate transcript).

To do this, I make a text file with all of the transcripts from open assistant and annotate them manually with brackets like this[[APT]] directly after the token it's tagging (or within it if it's not at a border because I don't always know the token boundaries).

Tags:
FIC
Fictional and truth-apt in the fictional world
LE
Truth-apt and a human could easily label it as true or false but hard
The thing that we will eventually be labeling is whether the tagged segment (sentence, bullet point, or clause) is true given the preceding context.
The reason we care about labeling is because we may want to train and evaluate ELK probes on these statements, and labels are necessary/helpful for this.
LH
Truth-apt and a human/LM could label it as true or false with difficulty
NORM
Truth-apt and makes a normative claim (about what is good or should be)
APT
Other truth-apt
Q
Question
STY
Syntactically truth-apt, but stylistic/idiomatic so it’s not quite truth-apt
COM
Command (not truth apt)
IMP
Implication of some truth-apt claim
FLAG
Flag this token as uncertain and needing to be revisited
CODE
Put this tag at the end of a code block (```Code block```[[CODE]] or `code line`[[CODE]])
Anything that is a programming language counts
# this is a comment #
If the [[ is immediately followed by # (like “I am an AI language model[[# this is a comment #,FIC]]”) then it is treated as a custom tag

Here are some examples:

## ORIGINAL ##

The "restrainer" is a person or entity that restrains or prohibits certain actions, such as evil behavior and sin. In Biblical theology, it is believed that there is an all-powerful God who has the ability to control and prevent everything in creation, including human beings' choices and actions. The Bible describes the restrainer as having the power to hold back the forces of evil and to guide humans on the right path.

This idea can be seen in the concept of simulations, which are computer systems designed to replicate and mimic real-world phenomena. In this regard, the idea of the restrainer can be seen as representing the presence of a higher power that controls the behavior of simulated entities like computers and robots. This suggests that while technology can provide opportunities for growth and development, it also may have the potential to limit or restrict individuals' freedom. Therefore, it is important to maintain a balance between technological advancement and ethical responsibility, ensuring that technology serves both individual and societal interests responsibly.


## ANNOTATED ##

The "restrainer" is a person[[APT]] or entity[[LE]] that restrains or prohibits certain actions[[LH]], such as evil behavior and sin[[LH]]. In Biblical theology, it is believed that there is an all-powerful God[[LE]] who has the ability to control and prevent everything in creation[[LH]], including human beings' choices and actions[[LH]]. The Bible describes the restrainer as having the power to hold back the forces of evil[[LH]] and to guide humans on the right path[[LH]].

This idea can be seen in the concept of simulations[[STY]], which are computer systems[[LE]] designed to replicate and mimic real-world phenomena[[APT]]. In this regard, the idea of the restrainer can be seen as representing the presence of a higher power[[APT]] that controls the behavior of simulated entities like computers and robots[[APT]]. This suggests that while technology can provide opportunities for growth and development, it also may have the potential to limit or restrict individuals' freedom[[APT]]. Therefore, it is important to maintain a balance between technological advancement and ethical responsibility[[APT, NORM]], ensuring that technology serves both individual and societal interests[[APT, NORM]] responsibly[[APT, NORM]].




## ORIGINAL ##

Berries are a great source of antioxidants, vitamins and minerals. They can help reduce inflammation, improve heart health, lower cholesterol levels, protect against certain cancers, support digestive health, boost immunity, and provide essential fatty acids.


## ANNOTATED ##

Berries are a great source of antioxidants[[LE]], vitamins and minerals[[LE]]. They can help reduce inflammation[[LE]], improve heart health[[LE]], lower cholesterol levels[[LE]], protect against certain cancers[[LE]], support digestive health[[LE]], boost immunity[[LE]], and provide essential fatty acids[[LE]].




## ORIGINAL ##

A Value-Added Tax (VAT) is a tax that is levied on goods and services at the time of sale, rather than when they are purchased. The main advantage of a VAT is that it eliminates the need for an income tax, as all revenue generated from the tax can be used to finance public spending.

The effects of abolishing the US income tax and replacing it with a VAT could vary depending on several factors, such as the specific design of the new tax system, its level of taxation, and its impact on economic growth and employment. Here's some possible scenarios:

Scenario 1: No change in overall taxes
Under this scenario, there would be no major changes to the existing tax system. Income taxes still exist, but their rates and brackets would remain unchanged. This could lead to a slight increase in total tax collections, as more people pay into the system through higher wages or profits. However, this would likely not have a significant effect on the overall economy.

Scenario 2: Lower overall taxes
If the new tax system resulted in lower overall tax collections compared to an income tax, it could have a positiveimpact on the economy. It could encourage businesses to create jobs and invest in the local community, as they may see reduced costs associated with operating in the United States. Additionally, individuals who previously avoided paying taxes due to low income might benefit from increased government assistance.

However, it's important to note that any reduction in taxes would also mean a decrease in government funding, which could negatively affect programs and policies designedto support vulnerable groups. Additionally, a lower overall tax rate might incentivize individuals and businesses to engage in activities that generate more taxableincome, potentially leading to unintended consequences such as increased inequality.

In summary, abolition ofthe US income tax and replacement with a VAT would significantly impact the country's economics, both positively and negatively. It's crucial to carefully consider the potential impacts ofsuch a policy before making any final decision.


## ANNOTATED ##

A Value-Added Tax (VAT) is a tax[[LE]] that is levied on goods and services[[LE]] at the time of sale[[LE]], rather than when they are purchased[[LE]]. The main advantage of a VAT is that it eliminates the need for an income tax[[LE]], as all revenue generated from the tax can be used to finance public spending[[LE]].

The effects of abolishing the US income tax and replacing it with a VAT could vary[[STY]] depending on several factors[[STY]], such as the specific design of the new tax system[[LE]], its level of taxation[[LE]], and its impact on economic growth[[LE]] and employment[[LE]]. Here's some possible scenarios[[STY]]:

Scenario 1: No change in overall taxes
Under this scenario, there would be no major changes to the existing tax system[[LE]]. Income taxes still exist[[LE]], but their rates and brackets would remain unchanged[[LE]]. This could lead to a slight increase in total tax collections[[LH]], as more people pay into the system[[LH]] through higher wages[[LH]] or profits[[LH]]. However, this would likely not have a significant effect on the overall economy[[LH]].

Scenario 2: Lower overall taxes
If the new tax system resulted in lower overall tax collections compared to an income tax, it could have a positiveimpact on the economy[[APT, NORM]]. It could encourage businesses to create jobs[[LE]] and invest in the local community[[LH]], as they may see reduced costs associated with operating in the United States[[LH]]. Additionally, individuals who previously avoided paying taxes due to low income might benefit[[APT, NORM]] from increased government assistance[[APT, NORM]].

However, it's important to note that any reduction in taxes would also mean a decrease in government funding[[APT, FLAG, NORM, LE]], which could negatively affect programs[[APT]] and policies designedto support vulnerable groups[[LH, NORM]]. Additionally, a lower overall tax rate might incentivize individuals and businesses to engage in activities[[APT]] that generate more taxableincome[[LH]], potentially leading to unintended consequences[[APT]] such as increased inequality[[LH]].

In summary, abolition ofthe US income tax and replacement with a VAT would significantly impact the country's economics[[LE]], both positively and negatively[[APT, NORM]]. It's crucial to carefully consider the potential impacts ofsuch a policy[[APT, NORM]] before making any final decision[[APT, NORM]].




## ORIGINAL ##

Donald Knuth is a famous computer scientist who has made many contributions to the field of computer science. He is best known for his work on the TeX typesetting system and the development of the C programming language.


## ANNOTATED ##

Donald Knuth is a famous computer scientist[[LE]] who has made many contributions[[LE]] to the field of computer science[[LE]]. He is best known for his work on the TeX typesetting system[[LH]] and the development of the C programming language[[LH]].




## ORIGINAL ##

Here is one way to write such a Python function:

``` 
def approx_frac(num): 
    \"\"\"Get approximation of a floating point number.\"\"\" 
    return (float(num) - num // 1e4)*1e-3 # approximately equivalent to floor() 

print("%.2f gives %.2f" % (.333333, aprox_frac(.333)))  # prints "0.334 gives 0.335" 

print("%.5f gives %.2f" % ((12.5, aprox_frac(12.5)),)) # prints "(12.50, 0.335)"
``` 
In this implementation, the `approx_frac` function takes a floating point number as input and returns an approximation to that value in the form of a fraction between 0 and 1. The function uses a simplified formula to calculate the approximation based on the difference between the actual number and its nearest integer multiple. Finally, the function provides some examples of how to use the function to get approximations of different values.


## ANNOTATED ##

Here is one way to write such a Python function[[STY]]:

``` 
def approx_frac(num): 
    \"\"\"Get approximation of a floating point number.\"\"\" 
    return (float(num) - num // 1e4)*1e-3 # approximately equivalent to floor() 

print("%.2f gives %.2f" % (.333333, aprox_frac(.333)))  # prints "0.334 gives 0.335" 

print("%.5f gives %.2f" % ((12.5, aprox_frac(12.5)),)) # prints "(12.50, 0.335)"
```[[CODE]] 
In this implementation, the `approx_frac` function takes a floating point number[[LE]] as input[[LE]] and returns an approximation to that value[[APT]] in the form of a fraction[[APT]] between 0 and 1[[LE]]. The function uses a simplified formula[[APT]] to calculate the approximation[[APT]] based on the difference between the actual number and its nearest integer multiple[[LE]]. Finally, the function provides some examples[[LE]] of how to use the function to get approximations[[LE]] of different values[[LE]].




## ORIGINAL ##

I am sorry, but I don't have access to such information as of yet.


## ANNOTATED ##

I am sorry[[APT]], but I don't have access to such information[[APT]] as of yet[[APT]].




## ORIGINAL ##

There are several advantages to answer set programming, or ASP. For example:

1- The ease of understanding and use for novices since it uses plain english instead of mathematical symbols that may not be easily understood by some programmers who do not have a strong background in math. 
2 - It is also faster compared to other programming languages because there exists an automated translation between programs written in plain English. This means you only need to write your program once but can then run this code many times without rewriting it each time. 

However you could say using python would give me more flexibility as I wouldn't need an engine to execute my code for instance


## ANNOTATED ##

There are several advantages to answer set programming[[STY]], or ASP[[STY]]. For example:

1- The ease of understanding[[IMP, LH]] and use[[IMP, LH]] for novices[[IMP, LH]] since it uses plain english[[IMP, LH]] instead of mathematical symbols[[IMP, LH]] that may not be easily understood[[IMP, LH]] by some programmers[[IMP, LH]] who do not have a strong background in math[[IMP, LH]]. 
2 - It is also faster compared to other programming languages[[LE]] because there exists an automated translation between programs written in plain English[[APT]]. This means you only need to write your program once[[LE]] but can then run this code many times[[LE]] without rewriting it[[LE]] each time[[LE]]. 

However you could say using python would give me more flexibility[[IMP, LE]] as I wouldn't need an engine[[LE]] to execute my code[[LE]] for instance[[LE]]




## ORIGINAL ##

Narnia is too violent. It should be nicer.

## ANNOTATED ##"""

In [12]:
inputs = tokenizer(prompt, return_tensors="pt", max_length=4096, truncation=True).to(device)
out = model.generate(**inputs, max_length=4096, do_sample=True, top_p=1, temperature=1, num_return_sequences=1)
print(tokenizer.decode(out[0], skip_special_tokens=True))


Goal
Make a classifier for whether a given piece of text can meaningfully be classified as true or false. More specifically, classify whether the the prefix of a document ending at a token t can be classified as true or false (whether it's “truth-apt”). Then we can use such a classifier to figure out which tokens to use an ELK probe on for a given piece of text (especially a debate transcript).

To do this, I make a text file with all of the transcripts from open assistant and annotate them manually with brackets like this[[APT]] directly after the token it's tagging (or within it if it's not at a border because I don't always know the token boundaries).

Tags:
FIC
Fictional and truth-apt in the fictional world
LE
Truth-apt and a human could easily label it as true or false but hard
The thing that we will eventually be labeling is whether the tagged segment (sentence, bullet point, or clause) is true given the preceding context.
The reason we care about labeling is because we may want

In [ ]:
# finetune the model to predict tags

# probably just copy-paste the weak_decept_ft.py code
from argparse import ArgumentParser
from collections import namedtuple
import json
from datasets import DatasetDict, load_dataset, Dataset
from itertools import islice
from peft import get_peft_model, LoraConfig, TaskType, PeftType
from popqa_meta_templates import templatize_ds
from sklearn.metrics import accuracy_score
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import default_data_collator, AutoModelForCausalLM, AutoTokenizer, get_linear_schedule_with_warmup
import numpy as np
import torch
import wandb
import time

wandb.login()


parser = ArgumentParser()
parser.add_argument("--model-name", type=str, default="EleutherAI/pythia-160m")
parser.add_argument("--ds-name", type=str, default="atmallen/TODO")
parser.add_argument("--max-length", type=int, default=4096)
parser.add_argument("--lr", type=float, default=2e-5)
parser.add_argument("--n-epochs", type=int, default=2)
parser.add_argument("--warmup-steps", type=int, default=400)
parser.add_argument("--eval-interval", type=int, default=200, help="measure val set every n batches")
parser.add_argument("--batch-size", type=int, default=8)
parser.add_argument("--weight-decay", type=float, default=0.1)
parser.add_argument("--n-train", type=int, default=-1)
parser.add_argument("--n-val", type=int, default=-1)
parser.add_argument("--lora-rank", type=int, default=32)
parser.add_argument("--lora-alpha", type=int, default=32)
parser.add_argument("--lora-dropout", type=float, default=0.1)
parser.add_argument("--device", type=str, default="cuda")
parser.add_argument("--no-peft", action="store_true")
parser.add_argument("--disable-cache", action="store_true")
parser.add_argument("--target-modules", nargs="+", default=["dense_h_to_4h", "dense_4h_to_h", "query_key_value"])
args_model_name = "EleutherAI/pythia-160m"
args_ds_name = "atmallen/TODO"
args_max_length = 4096
args_lr = 2e-5
args_n_epochs = 2
args_warmup_steps = 400
args_eval_interval = 200
args_batch_size = 8
args_weight_decay = 0.1
args_n_train = -1
args_n_val = -1
args_lora_rank = 32
args_lora_alpha = 32
args_lora_dropout = 0.1
args_device = "cuda"
args_no_peft = False
args_disable_cache = False
args_target_modules = ["dense_h_to_4h", "dense_4h_to_h", "query_key_value"]

# args = parser.parse_args()

model_name = args_model_name
ds_name = args_ds_name

max_length = args_max_length
lr = args_lr
num_epochs = args_n_epochs
warmup_steps = args_warmup_steps
eval_interval = args_eval_interval
batch_size = args_batch_size
weight_decay = args_weight_decay
n_train = args_n_train
n_val = args_n_val
lora_rank = args_lora_rank
lora_alpha = args_lora_alpha
lora_dropout = args_lora_dropout
device = args_device
use_peft = not args_no_peft
target_modules = args_target_modules
now = time.time()
save_name = f"custom-models/tagging/{model_name}-{ds_name}-{now}.pt"

# config for wandb
# cfg = vars(args)
# cfg["save_name"] = save_name

### LOAD/PROCESS DATASET, AND TRAIN MODEL ###

# load dataset
orig_ds = load_dataset(ds_name)

orig_ds["train"] = orig_ds["train"].shuffle()
orig_ds["validation"] = orig_ds["validation"].shuffle()

n_train = len(orig_ds["train"]) if n_train == -1 else n_train
n_val = len(orig_ds["validation"]) if n_val == -1 else n_val
ds = DatasetDict({
    "train": orig_ds["train"].select(range(n_train)),
    "validation": orig_ds["validation"].select(range(n_val)),
})

# instantiate tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# define templatize and tokenize functions
def tokenize_examples(examples):
    texts = examples["text"]

    # tokenize inputs
    inputs = tokenizer(texts, truncation=True, padding="max_length", max_length=max_length, return_tensors="pt", text_target=texts)

    maxlen = max([len(input_ids) for input_ids in inputs["input_ids"]])
    if maxlen > max_length:
        print(f"length: {maxlen} > max_length: {max_length} truncating")

    return inputs

# templateize and tokenize train
train_encodings = ds["train"].map(
    tokenize_examples,
    batched=True,
    num_proc=1,
    remove_columns=ds["train"].column_names,
    load_from_cache_file=not args_disable_cache,
    desc="Running tokenizer on dataset",
)
train_dataloader = DataLoader(
    train_encodings, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)

# validation and test
eval_encodings = ds["validation"].map(
    tokenize_examples,
    batched=True,
    num_proc=1,
    remove_columns=ds["train"].column_names,
    load_from_cache_file=not args_disable_cache,
    desc="Running tokenizer on dataset",
)
eval_dataloader = DataLoader(eval_encodings, collate_fn=default_data_collator, batch_size=1, pin_memory=True)

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
if use_peft:
    peft_config = LoraConfig(
        peft_type=PeftType.LORA, task_type=TaskType.CAUSAL_LM,
        inference_mode=False, target_modules=target_modules,
        r=lora_rank, lora_alpha=lora_alpha, lora_dropout=lora_dropout
    )
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
model = model.to(device)  # we want to keep the lora params in single precision, so don't call half() after pefting

# wandb.init(
#     project="weak-deception",
#     name=f"{'LoRA' if use_peft else 'ft'}-{model_name}-{ds_name}",
        
#     # track hyperparameters and run metadata
#     config=cfg
# )



def ids_to_text(ids):
    return tokenizer.batch_decode(ids, skip_special_tokens=True)


def eval_model(use_tqdm=False, dataloader=eval_dataloader):
    # for now just compute loss, in the future we might want to see precision and recall of tags
    model.eval()
    
    losses = []
    iterator = tqdm(dataloader) if use_tqdm else dataloader
    for batch in iterator:
        with torch.no_grad():
            batch = {k: v.to(device) for k, v in batch.items()}
            
            outputs = model(**batch)
            losses.append(outputs.loss.item())
            
    return namedtuple("EvalResults", ["loss"])(loss=np.mean(losses))

eval_result = eval_model(use_tqdm=True)
print(f"Initial Eval: {eval_result}")
train_eval_result = eval_model(use_tqdm=True, dataloader=train_dataloader)
print(f"Initial Train Eval: {train_eval_result}")

# only the LORA parameters should be updated
learnable_parameters = [p for p in model.parameters() if p.requires_grad]
print(f"Number of learnable parameters: {len(learnable_parameters)}")
optimizer = AdamW(learnable_parameters, lr=lr, weight_decay=weight_decay, betas=(0.9, 0.95))  # adam beta2 default is 0.999

lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=(len(train_dataloader) * num_epochs),
    )


for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        
        
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        if step % eval_interval == 0:
            eval_result = eval_model(use_tqdm=False)
            print(f"Eval: {eval_result}")
            train_eval_result = eval_model(use_tqdm=False, dataloader=train_dataloader)
            print(f"Train Eval: {train_eval_result}")
            # wandb.log({"train_acc": train_eval_result.acc, "train_acc_err": train_eval_result.acc_err, "train_acc_err_true": train_eval_result.true_acc_err, "train_acc_non_err": train_eval_result.acc_non_err,
            #            "acc": eval_result.acc, "acc_err": eval_result.acc_err, "acc_err_true": eval_result.true_acc_err, "acc_non_err": eval_result.acc_non_err,
            #            "perturbed_acc": perturbed_eval_result.acc, "perturbed_acc_err": perturbed_eval_result.acc_err, "perturbed_acc_err_true": perturbed_eval_result.true_acc_err, "perturbed_acc_non_err": perturbed_eval_result.acc_non_err,
            #            "train_loss": total_loss / step, "step": step, "epoch": epoch, "train_kl": kl, "pretraining_loss": pretraining_loss})

            model.train()
            
    print("Epoch {} loss: {}".format(epoch, total_loss / len(train_dataloader)))

# wandb.finish()

# save model
# this function is overridden by the peft library
model.save_pretrained(save_name)